make sure both processes get same audio


-choose file
- process via diarizer

In [1]:

%load_ext autoreload
%autoreload 2
import logging
from datetime import datetime, timezone

from app.database_redis.connection import get_redis_client
from app.services.apis.streamqueue_service.client import StreamQueueServiceAPI
from app.services.audio.redis import Connection, Diarizer, Meeting, Transcriber
from app.settings import settings
from app.services.audio.redis import Diarisation,Transcript
import json
import pandas as pd
import numpy as np

logger = logging.getLogger(__name__)

check_and_process_connections_interval_sec: 1.0


/tmp/ipykernel_147367/4266564517.py:12: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
redis_client = await get_redis_client(settings.redis_host, settings.redis_port, settings.redis_password)

In [3]:
meeting_id = 'xng-juvg-nyv'
diarization = Diarisation(meeting_id,redis_client=redis_client)
transcript = Transcript(meeting_id,redis_client=redis_client)

In [4]:
meeting = Meeting(redis_client,meeting_id)

In [5]:
def prep_transcripts(transcription):
    df = pd.DataFrame(transcription)
    df = df[[2,3,4]]
    df.columns = ['start','end','speech']
    df['start'] = pd.to_timedelta(df['start'],unit='s') + pd.Timestamp(seek)
    df['end'] = pd.to_timedelta(df['end'],unit='s') + pd.Timestamp(seek)
    
    return df

In [6]:
def prep_diarization(df):

    df['start'] = pd.to_timedelta(df['start'],unit='s')+pd.Timestamp(seek)
    df['end']   = pd.to_timedelta(df['end'],unit='s')+pd.Timestamp(seek)
    return df


In [7]:
def apply_diarization(trans_df,diar_df):
    segments = trans_df.to_dict('records')

    for seg in segments:
        diar_df['intersection'] = np.minimum(diar_df['end'], seg['end']) - np.maximum(diar_df['start'], seg['start'])
        speaker_ = diar_df[(diar_df['intersection'] == diar_df['intersection'].max())&(diar_df['intersection']>pd.Timedelta(0))]['speaker']
        if len(speaker_)>0:
            seg['speaker'] = speaker_.iloc[0]
            
    return pd.DataFrame(segments)

In [8]:
def prep_transcripts(new_transcriptions):
    dfs = []
    n=0
    for t,seek,connection_id in new_transcriptions:
        df = pd.DataFrame(t)[[2,3,4]]
        df.columns = ['start','end','speech']
        df['start'] = pd.to_timedelta(df['start'],unit='s') + pd.Timestamp(seek)
        df['end'] = pd.to_timedelta(df['end'],unit='s') + pd.Timestamp(seek)
        df['chunk']=n
        dfs.append(df)
        n+=1
        
    return pd.concat(dfs).reset_index()

In [9]:
def prep_diarizations(new_diarizations):
    dfs = []
    for d,seek,connection_id in new_diarizations:
        df = pd.DataFrame(d)
        df['start'] = pd.to_timedelta(df['start'],unit='s')+pd.Timestamp(seek)
        df['end']   = pd.to_timedelta(df['end'],unit='s')+pd.Timestamp(seek)
        dfs.append(df)
    return pd.concat(dfs)

In [10]:
def apply_diarization(trans_df,diar_df):
    segments = trans_df.to_dict('records')

    for seg in segments:
        diar_df['intersection'] = np.minimum(diar_df['end'], seg['end']) - np.maximum(diar_df['start'], seg['start'])
        speaker_ = diar_df[(diar_df['intersection'] == diar_df['intersection'].max())&(diar_df['intersection']>pd.Timedelta(0))]['speaker']
        if len(speaker_)>0:
            seg['speaker'] = speaker_.iloc[0]
            
    return pd.DataFrame(segments)

In [11]:

last_mutable = pd.DataFrame(columns = ['start','end','speech','speaker'])
immutable = pd.DataFrame(columns = ['start','end'])
mutable = pd.DataFrame(columns = ['start','end'])

In [15]:

new_diarizations = []
new_transcriptions = [] 
diar_df = pd.DataFrame(columns = ['start','end','speaker'])

In [16]:
pd.set_option('display.max_colwidth', None)

In [17]:

while True:
    d = await diarization.rpop()
    if not d:
        break
    new_diarizations.append(json.loads(d))
print(len(new_diarizations))

while True:
    d = await transcript.rpop()
    if not d:
        break
    new_transcriptions.append(json.loads(d))
print(len(new_transcriptions))
#TODO: we want to crop low sertainty 1/2 overlap from each side
if len(new_transcriptions)>0:
    trans_df = prep_transcripts(new_transcriptions) 
    diar_df = prep_diarizations(new_diarizations)
    trans_df = pd.concat([trans_df,last_mutable[last_mutable['speaker'].isna()]])
    mutable = apply_diarization(trans_df,diar_df)
    if len(mutable)>0:
        rename_dict = {s:n for n,s in enumerate(mutable['speaker'].dropna().unique().tolist())}
        mutable = mutable.replace(rename_dict).sort_values('start')
        immutable = pd.concat([immutable,last_mutable.dropna()]).reset_index(drop = True)
        last_mutable= mutable.copy()

            
    mutable=mutable.drop_duplicates('start')

    mutable['change'] = mutable['speaker']!=mutable['speaker'].shift()
    mutable['change'] = mutable['change'].cumsum()
mutable.groupby('change').agg({'speech':'sum','speaker':"first",'start':'first',"end":'last'})

1
2


/tmp/ipykernel_147367/4103374667.py:18: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  trans_df = pd.concat([trans_df,last_mutable[last_mutable['speaker'].isna()]])
/tmp/ipykernel_147367/4103374667.py:22: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  mutable = mutable.replace(rename_dict).sort_values('start')


,speech,speaker,start,end
change,,,,
1,А что теперь мне представить? Não desloou?,0,2024-05-25 16:49:17.944605+00:00,2024-05-25 16:49:24.142605+00:00
